In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import random
import string
import pandas as pd

# Creating Spark Session
spark = SparkSession.builder.appName('data_gen').getOrCreate()
spark.conf.set("spark.sql.sources.partitionOverwriteMode","dynamic")

24/07/01 02:08:29 WARN Utils: Your hostname, Ugurs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.24 instead (on interface en0)
24/07/01 02:08:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/01 02:08:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
weather = spark.read.parquet("/Users/ugurkalkavan/Downloads/m06sparkbasics/weather")

In [4]:
weather.select(spark_partition_id().alias("pid")).distinct().show()

+---+
|pid|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
| 13|
| 14|
| 15|
| 16|
| 17|
| 18|
| 19|
+---+
only showing top 20 rows



In [6]:
len(weather.inputFiles())

8796

In [15]:
weather.inputFiles()[0:4]

['file:///Users/ugurkalkavan/Downloads/m06sparkbasics/weather/year=2023/month=12/day=11/part-00214-44bd3411-fbe4-4e16-b667-7ec0fc3ad489.c000.snappy.parquet',
 'file:///Users/ugurkalkavan/Downloads/m06sparkbasics/weather/year=2018/month=07/day=21/part-00152-44bd3411-fbe4-4e16-b667-7ec0fc3ad489.c000.snappy.parquet',
 'file:///Users/ugurkalkavan/Downloads/m06sparkbasics/weather/year=2017/month=12/day=11/part-00214-44bd3411-fbe4-4e16-b667-7ec0fc3ad489.c000.snappy.parquet',
 'file:///Users/ugurkalkavan/Downloads/m06sparkbasics/weather/year=2020/month=02/day=19/part-00113-44bd3411-fbe4-4e16-b667-7ec0fc3ad489.c000.snappy.parquet']

In [115]:
from pyspark.sql.types import StructType, StructField, StringType



weather_df = spark.read.parquet("/Users/ugurkalkavan/Downloads/m06sparkbasics/weather").select("_metadata")

In [85]:
weather_df = weather_df.select("_metadata.file_path", "_metadata.file_name", "_metadata.file_size", "_metadata.file_block_start", "_metadata.file_block_length", "_metadata.file_modification_time")

In [93]:
weather_df = weather_df.select("_metadata.file_size", "_metadata.file_block_start", "_metadata.file_block_length", "_metadata.file_modification_time")

In [117]:
weather_df = weather_df.select("_metadata.file_path", "_metadata.file_name", "_metadata.file_size").dropDuplicates(["file_path","file_name"])


In [119]:
weather_df.count()

8796

In [120]:
weather_df.show(100,truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------+---------+
|file_path                                                                                                                                              |file_name                                                          |file_size|
+-------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------+---------+
|file:/Users/ugurkalkavan/Downloads/m06sparkbasics/weather/year=2016/month=05/day=29/part-00088-44bd3411-fbe4-4e16-b667-7ec0fc3ad489.c000.snappy.parquet|part-00088-44bd3411-fbe4-4e16-b667-7ec0fc3ad489.c000.snappy.parquet|6098626  |
|file:/Users/ugurkalkavan/Downloads/m06sparkbasics/weather/year=2016/mon

In [ ]:
estimate_df_size(weather)

In [5]:
import contextlib
import io
import re

from pyspark.sql import DataFrame

def _bytes2mb(bb: float) -> float:
  return bb / 1024 / 1024


def estimate_size_of_df(df: DataFrame, size_in_mb: bool = False) -> float:
  """Estimate the size in Bytes of the given DataFrame.
  If the size cannot be estimated return -1.0. It is possible if
  we failed to parse plan or, most probably, it is the case when statistics
  is unavailable. There is a problem that currently in the case of missing
  statistics spark return 8 (or 12) EiB. If your data size is really measured in EiB
  this function cannot help you. See https://github.com/apache/spark/pull/31817
  for details. Size is returned in Bytes!

  This function works only in PySpark 3.0.0 or higher!

  :param df: DataFrame
  :param size_in_mb: Convert output to Mb instead of B
  :returns: size in bytes (or Mb if size_in_mb)
  """
  with contextlib.redirect_stdout(io.StringIO()) as stdout:
      # mode argument was added in 3.0.0
      df.explain(mode="cost")

  # Get top line of Optimized Logical Plan
  # The output of df.explain(mode="cost") starts from the following line:
  # == Optimized Logical Plan ==
  # The next line after this should contain something like:
  # Statistics(sizeInBytes=3.0 MiB) (untis may be different)
  top_line = stdout.getvalue().split("\n")[1]

  # We need a pattern to parse the real size and untis
  pattern = r"^.*sizeInBytes=([0-9]+\.[0-9]+)\s(B|KiB|MiB|GiB|TiB|EiB).*$"

  _match = re.search(pattern, top_line)

  if _match:
      size = float(_match.groups()[0])
      units = _match.groups()[1]
  else:
      return -1

  if units == "KiB":
      size *= 1024

  if units == "MiB":
      size *= 1024 * 1024

  if units == "GiB":
      size *= 1024 * 1024 * 1024

  if units == "TiB":
      size *= 1024 * 1024 * 1024 * 1024

  if units == "EiB":
      # Most probably it is the case when Statistics is unavailable
      # In this case spark just returns max possible value
      # See https://github.com/apache/spark/pull/31817 for details
      size = -1

  if size < 0:
      return size

  if size_in_mb:
      return _bytes2mb(size)  # size in Mb

  return size  # size in bytes

In [11]:
estimate_size_of_df(weather,size_in_mb = False)

29742648524.8

In [ ]:
dir(weather)

In [1]:
def get_partitions_table(spark, table):
    from pyspark.sql.functions import regexp_replace,explode,expr
    
    # Get partition columns
    db, tbl = table.split(".")
    partitions = []
    Columns = spark.catalog.listColumns(tbl,db)
    partitionsColumns = list(filter(lambda spark: spark.isPartition, Columns))
    for pc in partitionsColumns:
        partitions.append(pc.name)

    if partitions:
        partitions_df = spark.sql(f"SHOW PARTITIONS {table}")
        from pyspark.sql.functions import split, concat
        partitions_df = partitions_df.withColumn("partition", split(partitions_df["partition"], "/"))


        select_exprs = [f"partition[{i}] as {partitions[i]}" for i in range(len(partitions))]
        partitions_df = partitions_df.selectExpr(*select_exprs)

        for p in partitions:
            partitions_df = partitions_df.withColumn(p, regexp_replace(partitions_df[p], f"{p}=", ""))
            
        return partitions_df
    else:
        print("THERE IS NO PARTITION IN THIS TABLE")
        
        return None

In [73]:
from pyspark.sql import Row

# Get list of input files
file_list = weather.inputFiles()

# Prepare a list to hold Row objects
rows = []

# Parse file paths to get partition info
for file_path in file_list:
    # Extract partition info from file path
    path_parts = file_path.split("/")
    partition_info = dict([part.split("=") for part in path_parts if "=" in part])
    # Create a Row from the partition info and append to list
    rows.append(Row(**partition_info))

# Create DataFrame from the list of Row objects
df_partitions = spark.createDataFrame(rows)

# Now, sort the DataFrame
sorted_df_partitions = df_partitions.sort(df_partitions.columns)

# Show the DataFrame
df_partitions.show()

sorted_df_partitions.show()

+----+-----+---+
|year|month|day|
+----+-----+---+
|2023|   12| 11|
|2018|   07| 21|
|2017|   12| 11|
|2020|   02| 19|
|2016|   06| 04|
|2023|   04| 10|
|2022|   04| 30|
|2017|   03| 26|
|2016|   01| 03|
|2023|   07| 24|
|2019|   09| 20|
|2022|   11| 03|
|2023|   10| 15|
|2023|   07| 20|
|2020|   10| 22|
|2022|   04| 17|
|2022|   08| 04|
|2016|   08| 31|
|2023|   02| 09|
|2023|   08| 30|
+----+-----+---+
only showing top 20 rows



+----+-----+---+
|year|month|day|
+----+-----+---+
|2016|   01| 01|
|2016|   01| 01|
|2016|   01| 01|
|2016|   01| 02|
|2016|   01| 02|
|2016|   01| 02|
|2016|   01| 03|
|2016|   01| 03|
|2016|   01| 03|
|2016|   01| 04|
|2016|   01| 04|
|2016|   01| 04|
|2016|   01| 05|
|2016|   01| 05|
|2016|   01| 05|
|2016|   01| 06|
|2016|   01| 06|
|2016|   01| 06|
|2016|   01| 07|
|2016|   01| 07|
+----+-----+---+
only showing top 20 rows



In [111]:

def get_partition_file_sizes(c, hdfs_path):
    hadoop = c._jvm.org.apache.hadoop
    fs = hadoop.fs.FileSystem
    conf = hadoop.conf.Configuration() 
    path = hadoop.fs.Path(hdfs_path)

    # Initialize variables
    total_files = 0
    total_size = 0

    # Iterate through files in the HDFS path
    for f in fs.get(conf).listStatus(path):
        file_name = f.getPath().getName()
        if file_name != "_SUCCESS":
            total_files += 1  # Increment file count
            total_size += f.getLen()  # Add file size

    # Calculate average file size
    if total_files > 0:
        average_size = total_size / total_files
    else:
        average_size = 0

    return total_files, total_size, average_size

In [151]:

def get_partition_file_sizes(spark: SparkSession, hdfs_path: str) -> tuple[int, int, int]:
    """
    Returns the total number of files, total file size,
    and average file sizes of partition files in a HDFS path.
    
    Parameters:
    spark: SparkSession object.
    hdfs_path: String that represents HDFS path.
      
    Returns:
    Tuple that contains total files, total size and average size.
    """    
    # Import the necessary Hadoop classes 
    hadoop = spark._jvm.org.apache.hadoop
    conf = hadoop.conf.Configuration() 
    path = hadoop.fs.Path(hdfs_path)

    # Initialize filesystem object
    fs = hadoop.fs.FileSystem.get(conf)
    
    # Initialize accumulator variables
    total_files = spark.sparkContext.accumulator(0)
    total_size = spark.sparkContext.accumulator(0)

    # Define recursive function to traverse file tree structure
    def file_tree_size(dir_path):
        for f in fs.listStatus(dir_path):
            if f.isFile():  # If file, accumulate size
                total_files.add(1)
                total_size.add(f.getLen())
            else:  # If directory, recurse
                file_tree_size(f.getPath())
                
    # Call recursive function starting at root
    file_tree_size(path)

    # Calculate average file size
    average_size = total_size.value / total_files.value if total_files.value > 0 else 0

    return total_files.value, total_size.value, average_size

In [169]:
get_partition_file_sizes(spark,
                         "file:///Users/ugurkalkavan/Downloads/m06sparkbasics/weather/year=2018/month=07/day=21/part-00152-44bd3411-fbe4-4e16-b667-7ec0fc3ad489.c000.snappy.parquet")

                         

(1, 1761355, 1761355.0)

In [171]:
get_partition_file_sizes(spark,
"file:///Users/ugurkalkavan/Downloads/m06sparkbasics/weather/year=2018/month=07/day=21/")



(3, 10200545, 3400181.6666666665)

In [160]:
get_partition_file_sizes(spark,
"file:///Users/ugurkalkavan/Downloads/m06sparkbasics/weather/year=2018/month=07/")



(91, 304450070, 3345605.164835165)

In [162]:
get_partition_file_sizes(spark,
                         "file:///Users/ugurkalkavan/Downloads/m06sparkbasics/weather/year=2018/month=07/day=21/part-00152-44bd3411-fbe4-4e16-b667-7ec0fc3ad489.c000.snappy.parquet")

                         

(1, 1761355, 1761355.0)

In [164]:
get_partition_file_sizes(spark,
"file:///Users/ugurkalkavan/Downloads/m06sparkbasics/weather/year=2018/month=07/day=21/")



(3, 10200545, 3400181.6666666665)

In [166]:
get_partition_file_sizes(spark,
"file:///Users/ugurkalkavan/Downloads/m06sparkbasics/weather/year=2018/month=07/")



(91, 304450070, 3345605.164835165)